### Code to read in HDF5 file and calculate the multiwavelength amplitude

In [ ]:
# Import the required packages 
import numpy as np
import h5py

# List to store the Galaxy IDs
galaxy_ids = [23205]

# Function to calculate the normalised A_m amplitude for a given m
def calculate_amplitudes(coscoefs, sincoefs, nmin, nmax, m):
    A_m = np.sqrt(np.sum(coscoefs[m, nmin:nmax+1]**2 + sincoefs[m, nmin:nmax+1]**2))
    A_0 = np.sqrt(np.sum(coscoefs[0, nmin:nmax+1]**2 + sincoefs[0, nmin:nmax+1]**2))
    return A_m / A_0

# Filter Names (JWST NIRCam + HST WFC3/ACS)
filters = ['F444W', 'F410M', 'F356W', 'F277W', 'F200W', 'F160W', 'F125W', 'F115W', 'F606W', 'F814W']

# Define the bounds for the radial and fourier modes
nmin = 0
nmax = 24
m = 2

# Define the number of realisations to average over. Max value = 100
num_realizations = 100

# Loop through each Galaxy ID
for galaxy_id in galaxy_ids:
    
    # Define the file name to read coefficients from
    file_path = f"{galaxy_id:05d}_error.hdf5"

    # Create a dictionary to store the amplitude values in different filters for each realization
    amplitude_values = {filter_name: [] for filter_name in filters}

    try:
        
        # Open the HDF5 file in a 'read only' mode
        with h5py.File(file_path, 'r') as f:
            
            # For each filter in the above defined list
            for filter_name in filters:
                
                # Pass the filter name all in lower case
                key = filter_name.lower()
                
                # Read in all the cos and sin coefficients 
                coscoefs = f[f'{key}/{galaxy_id}/expansion/coscoefs'][:]
                sincoefs = f[f'{key}/{galaxy_id}/expansion/sincoefs'][:]

                # For each realisation
                for realization in range(num_realizations):
                    
                    # Read in the specific array of coefficients 
                    coscoefs_realization = coscoefs[:, :, realization]
                    sincoefs_realization = sincoefs[:, :, realization]
                    
                    # Calculate the normalised amplitude  
                    amplitude = calculate_amplitudes(coscoefs_realization, sincoefs_realization, nmin, nmax, m)
                    
                    # Store the amplitude value with the respective filter name key
                    amplitude_values[filter_name].append(amplitude)
                    
        composite_dictionary = {}
        for filter_name, values in amplitude_values.items():
            
            # Calculating median and uncertainty for the F444W filter
            median_amplitude = np.median(values)
            error_amplitude = np.std(values)
            composite_dictionary[filter_name] = (median_amplitude, error_amplitude)
            
        # Calculate log of median asymmetry
        log_median_amplitude = np.log10(median_amplitude)
        
        # Calculate the upper and lower bounds of the error
        upper_error = np.log10(median_amplitude + error_amplitude) - log_median_amplitude
        lower_error = log_median_amplitude - np.log10(median_amplitude - error_amplitude)
    
    except Exception as e:
        print(f"Error processing galaxy ID {galaxy_id}: {e}")

### Code to reconstruct galaxy images from coefficients

In [ ]:
# Import the required packages 
import numpy as np
import h5py
import matplotlib.pyplot as plt
from scipy.special import eval_genlaguerre
from matplotlib.colors import LogNorm

# Function to read in the HDF5 file 
def readsnapshot(datafile, groupname):
    
    # Open the file in read only mode
    with h5py.File(datafile, 'r') as f:
        
        # Assuming the image pixels are stores in the 2 index
        image_array = f[groupname]['image'][2] 
        
        # Get the x and y dimensions of the image
        xdim, ydim = image_array.shape
        
        # Use this information to create a grid for plotting
        xpixels = np.linspace(-xdim / 2, xdim / 2, xdim)
        ypixels = np.linspace(-ydim / 2, ydim / 2, ydim)
        xpix, ypix = np.meshgrid(xpixels, ypixels, indexing='ij')
        
        # Set the radial and phi values
        rr, pp = np.sqrt(xpix**2 + ypix**2), np.arctan2(ypix, xpix)
        
        return rr, pp, xpix, ypix, image_array, xdim, ydim


# Laguerre Expansion Functions from the FLEX pipeline
def _gamma_n(nrange, rscl):
    return (rscl / 2.) * np.sqrt(nrange + 1.)

def _G_n(R, nrange, rscl):
    laguerre = np.array([
        eval_genlaguerre(n, 1, 2 * R / rscl) / _gamma_n(n, rscl)
        for n in nrange
    ])
    return np.exp(-R / rscl) * laguerre

def _n_m(mmax):
    nmvals = np.zeros(mmax)
    nmvals[0] = 1.0
    return np.power((nmvals + 1) * np.pi / 2., -0.5)

def laguerre_reconstruction(coscoefs, sincoefs, nmax, mmax, rr, pp, rscl):
    nmvals = _n_m(mmax)
    G_j = _G_n(rr, np.arange(nmax), rscl)
    
    fftotal = sum(
        coscoefs[m, n] * nmvals[m] * np.cos(m * pp) * G_j[n] +
        sincoefs[m, n] * nmvals[m] * np.sin(m * pp) * G_j[n]
        for m in range(mmax) for n in range(nmax)
    )
    return 0.5 * fftotal

# Function to reconstruct the laguerre expansions from coefficients
def perform_laguerre_reconstruction(datafile, galaxy_id, groupname, nmax, mmax):
    
    try:
        
        # Open the HDF5 file in read only mode
        with h5py.File(datafile, "r") as f:
            
            # Access the data in the file stored under various headers
            expansion = f[groupname][f'{galaxy_id}']['expansion']
            rscl = expansion.attrs['scale_length']
            center_x, center_y = expansion.attrs['centre']
            coscoefs, sincoefs = expansion[:]

        rr, pp, xpix, ypix, image_array, xdim, ydim = readsnapshot(datafile, groupname)
        reconstruction = laguerre_reconstruction(coscoefs, sincoefs, nmax, mmax, rr, pp, rscl)

        return {
            'reconstruction': reconstruction,
            'xpix': xpix, 'ypix': ypix,
            'xdim': xdim, 'ydim': ydim,
            'center_x': center_x, 'center_y': center_y,
            'image_array': image_array,
            'sincoefs': sincoefs,
            'coscoefs': coscoefs
            
        }

    except KeyError as e:
        print(f"Missing data for galaxy {galaxy_id}, group {groupname}: {e}")
        
    except Exception as e:
        print(f"General error in reconstruction: {e}")

def plot_reconstructions(results, filters):
    cval = np.linspace(-5., 1., 32)
    fig, axes = plt.subplots(2, len(filters), figsize=(4 * len(filters), 8), sharex=True, sharey=True)

    for i, f in enumerate(filters):
        try:
            d = results[f]
            recon_reshaped = d['reconstruction'].reshape(d['xdim'], d['ydim'])

            axes[0, i].contourf(d['xpix'], d['ypix'], np.log10(d['image_array']), cval, cmap='Greys')
            axes[0, i].set_title(f, fontsize=40)
            axes[0, i].axis('off')

            axes[1, i].contourf(
                d['xpix'] - d['center_x'], d['ypix'] - d['center_y'],
                np.log10(recon_reshaped), cval, cmap='Greys'
            )
            axes[1, i].axis('off')

        except Exception as e:
            print(f"Plotting error for {f}: {e}")

    plt.tight_layout()
    plt.show()

def plot_amplitude(results, filters, nmax, mmax):
    
    fig, axes = plt.subplots(1, len(filters), figsize=(5 * len(filters), 6), sharex=True, sharey=True)

    for i, f in enumerate(filters):
        try:
            d = results[f]
            sincoefs, coscoefs = d['sincoefs'], d['coscoefs']

            amplitudes = np.sqrt(sincoefs**2 + coscoefs**2).T

            axes[i].imshow(amplitudes, cmap='Blues', norm=LogNorm(), aspect='auto')
            axes[i].set_xlabel('Harmonics (m)')
            axes[i].set_ylabel('Radial Nodes (n)')
            axes[i].set_title(f, fontsize=40)
            axes[i].set_xticks(np.arange(mmax))
            axes[i].set_yticks(np.arange(nmax))
            axes[i].axis('off')

        except Exception as e:
            print(f"Plotting error for {f}: {e}")

    plt.tight_layout()
    plt.show()

# Execute the entire reconstruction and amplitude chart creation
if __name__ == "__main__":
    
    galaxy_ids = [23205]

    for galaxy_id in galaxy_ids: 
        datafile = f"EGS_{galaxy_id:05d}.hdf5"
        nmax, mmax = 24, 8
        filters = ['F606W', 'F814W', 'F115W', 'F125W', 'F160W',
                   'F200W', 'F277W', 'F356W', 'F410M', 'F444W']

        results = {}
        for filt in filters:
            res = perform_laguerre_reconstruction(datafile, galaxy_id, filt.lower(), nmax, mmax)
            if res:
                results[filt] = res

        plot_reconstructions(results, filters)
        plot_amplitude(results, filters, nmax, mmax)